<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Neural-Network" data-toc-modified-id="Neural-Network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Neural Network</a></span></li><li><span><a href="#Ok-cool" data-toc-modified-id="Ok-cool-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ok cool</a></span></li></ul></div>

In [7]:
import torch

device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device)
w2 = torch.randn(H, D_out, device=device)

In [8]:
learning_rate = 1e-6
for t in range(500):
  # Forward pass: compute predicted y
  h = x.mm(w1)
  h_relu = h.clamp(min=0)
  y_pred = h_relu.mm(w2)

  # Compute and print loss; loss is a scalar, and is stored in a PyTorch Tensor
  # of shape (); we can get its value as a Python number with loss.item().
  loss = (y_pred - y).pow(2).sum()
  print(t, loss.item())

  # Backprop to compute gradients of w1 and w2 with respect to loss
  grad_y_pred = 2.0 * (y_pred - y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h < 0] = 0
  grad_w1 = x.t().mm(grad_h)

  # Update weights using gradient descent
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

0 37289116.0
1 37908056.0
2 43386976.0
3 44586468.0
4 35570444.0
5 20559140.0
6 9378759.0
7 4132521.5
8 2150955.25
9 1388170.25
10 1038436.5625
11 836493.4375
12 696987.25
13 590481.3125
14 505298.75
15 435625.1875
16 377842.03125
17 329468.28125
18 288575.84375
19 253798.46875
20 224034.59375
21 198461.765625
22 176409.4375
23 157289.453125
24 140648.46875
25 126103.375
26 113348.140625
27 102120.171875
28 92200.765625
29 83411.5703125
30 75604.4765625
31 68645.703125
32 62428.98046875
33 56864.5625
34 51884.5078125
35 47415.15625
36 43385.421875
37 39746.76953125
38 36458.9453125
39 33481.1875
40 30778.48828125
41 28322.673828125
42 26088.13671875
43 24052.21484375
44 22194.03125
45 20495.939453125
46 18943.943359375
47 17523.84375
48 16222.1083984375
49 15027.3486328125
50 13929.9267578125
51 12921.1875
52 11995.453125
53 11143.359375
54 10357.5263671875
55 9632.2626953125
56 8962.767578125
57 8344.15625
58 7771.82666015625
59 7241.7421875
60 6751.2353515625
61 6296.91650390625
62 5

## Neural Network

In [23]:
# Code in file nn/two_layer_net_module.py
import torch

class TwoLayerNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.
    """
    super(TwoLayerNet, self).__init__()
    self.linear1 = torch.nn.Linear(D_in, H)
    self.linear2 = torch.nn.Linear(H, D_out)

  def forward(self, x):
    """
    In the forward function we accept a Tensor of input data and we must return
    a Tensor of output data. We can use Modules defined in the constructor as
    well as arbitrary (differentiable) operations on Tensors.
    """
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    return y_pred

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above.
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(50000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = loss_fn(y_pred, y)
    if t % 10000 == 0:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 1.0581629276275635
10000 0.4231279790401459
20000 0.17353735864162445
30000 0.0670170933008194
40000 0.026668930426239967


In [29]:
model(x[63])

tensor([-0.0037, -0.1132,  0.5343, -0.6003, -0.6639,  1.6061,  0.6295,
         0.4072,  1.0311,  0.7904])

In [30]:
y[63]

tensor([ 0.0302, -0.1166,  0.5118, -0.5859, -0.6820,  1.5857,  0.5887,
         0.3936,  0.9835,  0.7885])

This looks very good. Let's go deeper.

In [31]:
# Code in file nn/two_layer_net_module.py
import torch

class TwoLayerNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.
    """
    super(TwoLayerNet, self).__init__()
    self.linear1 = torch.nn.Linear(D_in, H)
    self.linear2 = torch.nn.Linear(H, D_out)

  def forward(self, x):
    """
    In the forward function we accept a Tensor of input data and we must return
    a Tensor of output data. We can use Modules defined in the constructor as
    well as arbitrary (differentiable) operations on Tensors.
    """
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    return y_pred

In [ ]:
# torch.from_numpy([])

In [44]:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 10, 100, 10

# Create random Tensors to hold inputs and outputs
# x = torch.randn(N, D_in)
# y = torch.randn(N, D_out)

x = torch.from_numpy([])

# Construct our model by instantiating the class defined above.
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(100000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = loss_fn(y_pred, y)
    if t % 10000 == 0:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(t, loss.item())

0 0.9693735241889954
10000 0.6084838509559631
20000 0.4031957983970642
30000 0.27504071593284607
40000 0.19393056631088257
50000 0.1413998305797577
60000 0.10629907995462418
70000 0.082027368247509
80000 0.06474994868040085
90000 0.05214588716626167
99999 0.042807869613170624


In [42]:
model(x[63])

tensor([-0.2524, -0.6917,  0.9070, -0.3784,  0.3710, -0.2966, -0.3182,
        -0.6720,  0.8225,  0.5203])

In [45]:
x[63]

tensor([-0.4218, -2.0758, -0.3345, -0.3585, -0.5878,  0.8537,  0.8468,
         0.5459,  1.0140, -0.8848])

## Ok cool

Now let's create a seconds to categorical periods transformation function.

In [81]:
import datetime
import math

In [1694]:
a = round(datetime.datetime.now().timestamp())
prev = a
for i in range(1,10):
#     print(prev,  (prev % ((4**i))), '%', i, (2**i))

    print ( math.floor(prev % (4**i) / (4**i) * 4) , '%', (4**i))

#     prev -= prev % (4**i)

3 % 4
0 % 16
1 % 64
0 % 256
2 % 1024
3 % 4096
0 % 16384
3 % 65536
3 % 262144
